In [2]:
import urllib

from google.colab import drive

drive.mount('/content/drive')

!pip install -q -U transformers==4.37.2
!pip install -q bitsandbytes==0.41.3 accelerate==0.25.0
!pip install datasets
!pip install openai

import requests
from PIL import Image
from datasets import *
dataset = load_dataset("AILab-CVC/SEED-Bench")
from itertools import islice
import copy

import openai
import os
from openai import OpenAI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency res

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for AILab-CVC/SEED-Bench contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/AILab-CVC/SEED-Bench
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.


Generating test split: 0 examples [00:00, ? examples/s]

In [14]:
dims = [1, 2, 3, 4, 5, 6, 7, 8, 9]
dic = {key: 0 for key in dims}

def transform_dicts(data_list):
    new_list = []
    for item in data_list:

        if int(item['question_type_id']) > 9:
            continue
        dic[int(item['question_type_id'])] += 1
        if dic[int(item['question_type_id'])] > 10:
            continue
        new_dict = item.copy()  # Copy the original dictionary
        new_list.append(new_dict)
    return new_list


# def transform_dicts(data_list):
#     new_list = []
#     for item in data_list:

#         if int(item['question_type_id']) > 9:
#             continue
#         dic[int(item['question_type_id'])] += 1
#         if dic[int(item['question_type_id'])] > 15:
#             continue
#         for choice_key in ['choice_a', 'choice_b', 'choice_c', 'choice_d']:
#             new_dict = item.copy()  # Copy the original dictionary
#             new_dict['answer'] = 'T' if item['answer'].lower() == choice_key[-1] else 'F'
#             new_dict['choice'] = item[choice_key]  # Set the single choice to the value from the original dict
#             new_dict.pop('choice_a', None)
#             new_dict.pop('choice_b', None)  # Remove other choice keys
#             new_dict.pop('choice_c', None)
#             new_dict.pop('choice_d', None)
#             new_list.append(new_dict)
#     return new_list

tf_dataset = copy.deepcopy(dataset)
tf_dataset['test'] = transform_dicts(dataset['test'])

In [15]:
tf_dataset['test'][0]

{'answer': 'A',
 'choice_a': 'One',
 'choice_b': 'Two',
 'choice_c': 'Three',
 'choice_d': 'Four',
 'data_id': '1454426_2591111986',
 'data_type': 'image',
 'question': 'How many towels are in the image?',
 'question_id': '101669',
 'question_type_id': '5',
 'segment': None}

In [21]:
MODEL = "4o"
client = OpenAI(
    api_key = "sk-proj-ZRb1n9aFEihYMCSSdHdwT3BlbkFJmpH8AM0Xn3IsJUxztTtB",
)

def split_question(question, options, correct):
    completion = client.chat.completions.create(
    model=f"gpt-{MODEL}",
    messages=[
        {"role": "system", "content":
"""Given a multiple-choice question with 4 choices and the answer, rephrase the question into a sequence of 2 binary questions, where each question eliminates half the choices. Return your response in JSON form as follows:
{
    "1":<question 1>,
    "1correct": The correct answer for 1 'Yes' or 'No',
    "2":<question 2>,
    "2correct": The correct answer for 2 'Yes' or 'No'
}
Return nothing except the JSON, not even code block brackets
"""},
        {"role": "user", "content": f"QUESTION:{question}\nOPTIONS:{options}\nANSWER:{correct}"}
    ]
    )
    result = completion.choices[0].message.content
    print(result)
    return result

    # result_list = result.split("\n")
    # # remove every line before and after the code
    # for i in range(len(result_list)):
    #     if "```" in result_list[i]:
    #         result_list = result_list[i+1:]
    #         break

    # for i in range(len(result_list)):
    #     if "```" in result_list[i]:
    #         result_list = result_list[:i]
    #         break

    # recombine the lines
    # result = result_list

    # pipe the result into a file
    # with open(filename, "w") as f:
    #     f.write(code)
    #     f.write("\n".join(result))


In [7]:
tf_dataset['test'][0]

{'answer': 'A',
 'choice_a': 'One',
 'choice_b': 'Two',
 'choice_c': 'Three',
 'choice_d': 'Four',
 'data_id': '1454426_2591111986',
 'data_type': 'image',
 'question': 'How many towels are in the image?',
 'question_id': '101669',
 'question_type_id': '5',
 'segment': None}

In [22]:
temp = tf_dataset['test'][0]
choices = [temp['choice_a'], temp['choice_b'], temp['choice_c'], temp['choice_d']]
correct = ''
if temp['answer'] == 'A':
    correct = temp['choice_a']
if temp['answer'] == 'B':
    correct = temp['choice_b']
if temp['answer'] == 'C':
    correct = temp['choice_c']
if temp['answer'] == 'D':
    correct = temp['choice_d']

res = split_question(tf_dataset['test'][0]['question'], choices, correct)

{
    "1": "Is the number of towels one of the first two options: One or Two?",
    "1correct": "Yes",
    "2": "Is the number of towels One?",
    "2correct": "Yes"
}


In [18]:
import json
res = json.loads(res)
res

{'1': 'Is the number of towels one or two?',
 '1correct': 'Yes',
 '2': 'Is the number of towels one?',
 '2correct': 'Yes'}

In [23]:
revised_dataset = []
for i in range(len(tf_dataset['test'])):
    print(f"current question: {i}")
    print()

    temp = tf_dataset['test'][i]
    q1 = temp.copy()
    q2 = temp.copy()

    choices = [temp['choice_a'], temp['choice_b'], temp['choice_c'], temp['choice_d']]
    correct = ''
    if temp['answer'] == 'A':
        correct = temp['choice_a']
    if temp['answer'] == 'B':
        correct = temp['choice_b']
    if temp['answer'] == 'C':
        correct = temp['choice_c']
    if temp['answer'] == 'D':
        correct = temp['choice_d']

    res = split_question(tf_dataset['test'][i]['question'], choices, correct)
    res = json.loads(res)
    q1['question'] = res['1']
    q1['answer'] = res['1correct']
    q1.pop('choice_a', None)
    q1.pop('choice_b', None)  # Remove other choice keys
    q1.pop('choice_c', None)
    q1.pop('choice_d', None)
    revised_dataset.append(q1)

    q2['question'] = res['2']
    q2['answer'] = res['2correct']
    q2.pop('choice_a', None)
    q2.pop('choice_b', None)  # Remove other choice keys
    q2.pop('choice_c', None)
    q2.pop('choice_d', None)
    revised_dataset.append(q2)

current question: 0

{
    "1": "Are there two or more towels in the image?",
    "1correct": "No",
    "2": "Is there only one towel in the image?",
    "2correct": "Yes"
}
current question: 1

{
    "1": "Is the building a hotel or a house?",
    "1correct": "No",
    "2": "Is the building a cabin?",
    "2correct": "Yes"
}
current question: 2

{
    "1":"Is the man using a device?",
    "1correct": "Yes",
    "2":"Is the man using the device to capture an image?",
    "2correct": "Yes"
}
current question: 3

{
    "1": "Is the chair on the right side of the image either White or Black?",
    "1correct": "Yes",
    "2": "Is the chair on the right side of the image White?",
    "2correct": "Yes"
}
current question: 4

{
    "1": "Is the number of people in the image greater than two?",
    "1correct": "Yes",
    "2": "Is the number of people in the image four?",
    "2correct": "Yes"
}
current question: 5

{
    "1": "Are there more than two men on the airplane?",
    "1correct": "No"

In [25]:
len(revised_dataset)

180

In [26]:
import json
# make new JSON file
with open('/content/drive/My Drive/BinarySEEDBenchNew.json', 'w') as f:
    json.dump(revised_dataset, f)